In [ ]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tree import Tree
import pandas as pd
from transformers import pipeline
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

# Named Entity Recognition

## NER Utilizando métodos tradicionales

Los métodos "tradicionales" son más eficientes aunque pueden no ser tan efectivos como modelos de lenguaje para dominios específicos. Estos métodos usan una combinacion de POS y diccionarios para la identificación de NER.

In [ ]:
# Descargar los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# Crear un conjunto de textos para analizar
texts = [
    "Apple Inc. is looking to buy a startup in the United States.",
    "Barack Obama was the 44th President of the United States.",
    "Elon Musk founded SpaceX, an aerospace manufacturer and space transportation company.",
    "The Amazon rainforest is the largest tropical rainforest in the world.",
    "Google was founded by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University."
]

# Crear un DataFrame
texts_df = pd.DataFrame(texts, columns=['Text'])


En este caso se utiliza la función `ne_chunk` de `nltk` para la identificación de NER

In [ ]:
# Función para extraer entidades nombradas
def extract_entities(text):
    # Tokenizar el texto
    words = word_tokenize(text)
    # Etiquetado de partes del discurso
    pos_tags = pos_tag(words)
    # Reconocimiento de entidades nombradas
    chunks = ne_chunk(pos_tags)
    entities = []
    
    for chunk in chunks:
        if isinstance(chunk, Tree):
            entity = " ".join([token for token, pos in chunk.leaves()])
            entity_type = chunk.label()
            entities.append((entity, entity_type))
    
    return entities

In [ ]:
%%time

# Aplicar el reconocimiento de entidades nombradas
texts_df['Entities'] = texts_df['Text'].apply(extract_entities)


texts_df.head()


# NER Utilizando modelos de lenguaje

Este ejemplo demuestra cómo aplicar modelos de lenguaje avanzados para NER. Suelen ser más precisos a un mayor costo computacional tanto a la hora de ejecutarse y ni que decir a la hora de entrenarse.

Cargamos los textos en el dataframe de nuevo:

In [ ]:
# Crear un DataFrame
texts_df = pd.DataFrame(texts, columns=['Text'])


Utilizamos el modelo `dbmdz/bert-large-cased-finetuned-conll03-english` obtenido de HuggingFace en [dbmdz/bert-large-cased-finetuned-conll03-english](https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).

Utilizamos la acceleración de la máquina, en este caso el motor mps en macOS ARM:

In [ ]:
%%time

# Cargar el modelo de reconocimiento de entidades nombradas
mps_device = "mps"
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple", device = mps_device)

# Función para extraer entidades nombradas
def extract_entities(text):
    entities = ner_pipeline(text)
    return [(entity['entity_group'], entity['word']) for entity in entities]

# Aplicar el reconocimiento de entidades nombradas
texts_df['Entities'] = texts_df['Text'].apply(extract_entities)

# Mostrar los resultados
# import ace_tools as tools; tools.display_dataframe_to_user(name="Named Entity Recognition", dataframe=texts_df)

texts_df.head()


Nótese la diferencia en tiempos de ejecución entre ambos ejemplos